### Combine

Combine is a Beam transform for combining collections of elements or values in your data. Combine has variants that work on entire PCollections, and some that combine the values for each key in PCollections of key/value pairs.

When you apply a Combine transform, you must provide the function that contains the logic for combining the elements or values. The combining function should be **commutative** and **associative**, as the function is not necessarily invoked exactly once on all values with a given key. Because the input data (including the value collection) may be distributed across multiple workers, the combining function might be called multiple times to perform partial combining on subsets of the value collection. The Beam SDK also provides some pre-built combine functions for common numeric combination operations such as sum, min, and max.

In [12]:
import apache_beam as beam

class PercentagesFn(beam.CombineFn):
    def create_accumulator(self):
        return {}

    def add_input(self, accumulator, input):
        # accumulator == {}
        # input == '🥕'
        if input not in accumulator:
            accumulator[input] = 0  # {'🥕': 0}
        accumulator[input] += 1  # {'🥕': 1}
        return accumulator

    def merge_accumulators(self, accumulators):
        # accumulators == [
        #     {'🥕': 1, '🍅': 2},
        #     {'🥕': 1, '🍅': 1, '🍆': 1},
        #     {'🥕': 1, '🍅': 3},
        # ]
        merged = {}
        for accum in accumulators:
            for item, count in accum.items():
                if item not in merged:
                    merged[item] = 0
                merged[item] += count
        # merged == {'🥕': 3, '🍅': 6, '🍆': 1}
        return merged

    def extract_output(self, accumulator):
        # accumulator == {'🥕': 3, '🍅': 6, '🍆': 1}
        total = sum(accumulator.values())  # 10
        percentages = {item: count / total for item, count in accumulator.items()}
        # percentages == {'🥕': 0.3, '🍅': 0.6, '🍆': 0.1}
        return percentages

with beam.Pipeline() as pipeline:
    percentages = (
        pipeline
        | 'Create produce' >> beam.Create(['🥕', '🍅', '🍅', '🥕', '🍆', '🍅', '🍅', '🍅', '🥕', '🍅'])
        | 'Get percentages' >> beam.CombineGlobally(PercentagesFn())
        | beam.Map(print))

{'🥕': 0.3, '🍅': 0.6, '🍆': 0.1}


In [21]:
import apache_beam as beam

class AverageFn(beam.CombineFn):
    def create_accumulator(self):
        return (0.0, 0)   # initialize (sum, count)

    def add_input(self, sum_count, input):
        (sum, count) = sum_count
        return sum + input, count + 1

    def merge_accumulators(self, accumulators):
        ind_sums, ind_counts = zip(*accumulators) # zip - [(27, 3), (39, 3), (18, 2)]  -->  [(27,39,18), (3,3,2)]
        return sum(ind_sums), sum(ind_counts)     # (84,8)

    def extract_output(self, sum_count):    
        (sum, count) = sum_count    # combine globally using CombineFn
        return sum / count if count else float('NaN')
    
with beam.Pipeline() as pipeline:
    percentages = (
        pipeline
        | beam.Create([15,5,7,7,9,23,13,5])
        | "Combine Globally" >> beam.CombineGlobally(AverageFn()) 
        
        # | 'Write results' >> beam.io.WriteToText('output/combine')
        | beam.Map(print)
    )

10.5
